In [10]:
!pip install evaluate
!pip install transformers
!pip install datasets

In [11]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
import evaluate

In [12]:
pss_data = pd.read_csv('/content/subset_file_50000_100000.csv')

# Display first few rows of the dataset
pss_data.head()

,pdb_id,chain_code,seq,sst8,sst3,len,has_nonstd_aa
0,5XFV,A,MAHHHHHHSAALEVLFQGPGSMSLKVNILGHEFSNPFMNAAGVLCT...,CCCCCCCCCCCCCCCCCCCCCCCCCEEETTEEESSSEEECTTSSCS...,CCCCCCCCCCCCCCCCCCCCCCCCCEEECCEEECCCEEECCCCCCC...,334,False
1,4R9X,A,MLEVIATCLEDVKRIERAGGKRIELISSYTEGGLTPSYAFIKKAVE...,CEEEEESSHHHHHHHHHTTCCEEEECBCGGGTCBCCCHHHHHHHHH...,CEEEEECCHHHHHHHHHCCCCEEEECECHHHCCECCCHHHHHHHHH...,233,False
2,5VAX,A,GSMAHAGRTGYDNREIVMKYIHYKLSQRGYEWDDGDDVEENRTEAP...,CCCCCCCCCCCCHHHHHHHHHHHHHHTTTCCCCCCCCCCCCCCCCC...,CCCCCCCCCCCCHHHHHHHHHHHHHHCCCCCCCCCCCCCCCCCCCC...,168,False
3,5LSJ,D,MGTLQKCFEDSNGKASDFSLEASVAEMKEYITKFSLERQTWDQLLL...,CCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHH...,CCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHH...,178,False
4,2J5I,B,MSTYEGRWKTVKVEIEDGIAFVILNRPEKRNAMSPTLNREMIDVLE...,CCCCCCCCSSEEEEEETTEEEEEECCGGGTTCBCHHHHHHHHHHHH...,CCCCCCCCCCEEEEEECCEEEEEECCHHHCCCECHHHHHHHHHHHH...,276,False


In [44]:
# Data augmention by oversampling the minority class
# Separate the dataset into different classes
# coil_data = pss_data[pss_data['sst3'] == 'C']
# sheet_data = pss_data[pss_data['sst3'] == 'E']
# helix_data = pss_data[pss_data['sst3'] == 'H']

# Determine the maximum count among the classes (the majority class)
# max_class_size = max(coil_data.shape[0], sheet_data.shape[0], helix_data.shape[0])

# Oversample the minority classes to match the size of the majority class
# Check if the dataframes are not empty before sampling
# sheet_data_oversampled = sheet_data.sample(max_class_size, replace=True, random_state=42) if not sheet_data.empty else pd.DataFrame(columns=sheet_data.columns)
# helix_data_oversampled = helix_data.sample(max_class_size, replace=True, random_state=42) if not helix_data.empty else pd.DataFrame(columns=helix_data.columns)

# Combine the data back into one balanced dataset
# balanced_data = pd.concat([coil_data, sheet_data_oversampled, helix_data_oversampled])

# Shuffle the balanced dataset
# balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the new class distribution
# balanced_distribution = balanced_data['sst3'].value_counts()
# print(balanced_distribution)

# Removed data augmentation as it was causing issues with empty dataframes.
# Proceeding with original data split.
print("Data augmentation step removed.")

Data augmentation step removed.


In [14]:
pss_data_cleaned = pss_data.dropna(subset=['sst3'])

X = pss_data_cleaned['seq']  # Amino acid sequence
y = pss_data_cleaned['sst3']  # Secondary structure labels (C, E, H)

train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size=0.2, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size=0.5, random_state=42)

In [51]:
from datasets import Dataset
import torch
import torch.nn as nn

# Tokenizer (We won't use a pre-trained model, just padding and truncation)
def tokenize_data(texts, max_length=128):
    # Convert sequences into tokenized format (just numeric encoding here)
    tokenized_texts = []
    for seq in texts:
        # Use ASCII value for each amino acid as tokens
        tokenized_seq = [ord(c) for c in seq]
        # Pad or truncate the sequence to max_length
        if len(tokenized_seq) > max_length:
            tokenized_seq = tokenized_seq[:max_length]
        elif len(tokenized_seq) < max_length:
            tokenized_seq = tokenized_seq + [0] * (max_length - len(tokenized_seq)) # Pad with 0

        tokenized_texts.append(tokenized_seq)
    return tokenized_texts

# Define label map
label_map = {'C': 0, 'E': 1, 'H': 2}

# Filter sequences and labels simultaneously, character by character
def filter_data(texts, labels, label_map, max_length=128):
    filtered_texts = []
    filtered_labels = []
    for text, label_seq in zip(texts, labels):
        filtered_text_seq = []
        filtered_label_seq = []
        for i in range(min(len(text), len(label_seq), max_length)):
            if label_seq[i] in label_map:
                filtered_text_seq.append(ord(text[i]))
                filtered_label_seq.append(label_map[label_seq[i]])

        if filtered_text_seq: # Only add if there are valid tokens after filtering
            # Pad or truncate the filtered sequence and labels
            if len(filtered_text_seq) < max_length:
                filtered_text_seq += [0] * (max_length - len(filtered_text_seq))
                filtered_label_seq += [-100] * (max_length - len(filtered_label_seq)) # Use -100 for padding labels
            elif len(filtered_text_seq) > max_length:
                filtered_text_seq = filtered_text_seq[:max_length]
                filtered_label_seq = filtered_label_seq[:max_length]

            filtered_texts.append(filtered_text_seq)
            filtered_labels.append(filtered_label_seq)

    return filtered_texts, filtered_labels


# Filter the datasets
train_encodings, train_labels_filtered = filter_data(train_texts, train_labels, label_map)
val_encodings, val_labels_filtered = filter_data(val_texts, val_labels, label_map)
test_encodings, test_labels_filtered = filter_data(test_texts, test_labels, label_map)


# Print sizes after filtering and tokenization
print(f"Size of train_encodings: {len(train_encodings)}")
print(f"Size of train_labels_filtered: {len(train_labels_filtered)}")
print(f"Size of val_encodings: {len(val_encodings)}")
print(f"Size of val_labels_filtered: {len(val_labels_filtered)}")
print(f"Size of test_encodings: {len(test_encodings)}")
print(f"Size of test_labels_filtered: {len(test_labels_filtered)}")


# Convert the data into Hugging Face Dataset format with torch tensors
# Rename columns to match Trainer's expected input names
train_data = Dataset.from_dict({
    "input_ids": torch.tensor(train_encodings),
    "labels": torch.tensor(train_labels_filtered, dtype=torch.long),
})

val_data = Dataset.from_dict({
    "input_ids": torch.tensor(val_encodings),
    "labels": torch.tensor(val_labels_filtered, dtype=torch.long),
})

test_data = Dataset.from_dict({
    "input_ids": torch.tensor(test_encodings),
    "labels": torch.tensor(test_labels_filtered, dtype=torch.long),
})

# Print sizes of the created datasets
print(f"Size of train_data: {len(train_data)}")
print(f"Size of val_data: {len(val_data)}")
print(f"Size of test_data: {len(test_data)}")

Size of train_encodings: 35973
Size of train_labels_filtered: 35973
Size of val_encodings: 4497
Size of val_labels_filtered: 4497
Size of test_encodings: 4497
Size of test_labels_filtered: 4497
Size of train_data: 35973
Size of val_data: 4497
Size of test_data: 4497


In [52]:
# Define BiLSTM Model
class BiLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers=2, dropout=0.5):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(input_dim, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers=n_layers, dropout=dropout, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # *2 for bidirectional
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids, labels=None): # Modify forward method
        embedded = self.dropout(self.embedding(input_ids))
        lstm_out, _ = self.lstm(embedded)
        output = self.fc(lstm_out)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            # Adjust dimensions for loss calculation
            # Flatten the output and labels for CrossEntropyLoss
            loss = loss_fct(output.view(-1, self.fc.out_features), labels.view(-1))

        return {"loss": loss, "logits": output} # Return dictionary

In [53]:
# Initialize the BiLSTM model
model = BiLSTM(input_dim, hidden_dim, output_dim)

# Training Arguments
training_args = TrainingArguments(
    output_dir="./bilstm_output",
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="none",
    remove_unused_columns=False, # Add this line
)

In [54]:
# Evaluation Metrics
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
accuracy_metric = evaluate.load("accuracy")

# Define the compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Flatten the predictions and labels and remove padding (label -100)
    flat_predictions = predictions.flatten()
    flat_labels = labels.flatten()

    # Compute Accuracy, Precision, Recall, F1, and Macro F1 on filtered data
    accuracy = accuracy_metric.compute(predictions=flat_predictions, references=flat_labels)
    precision = precision_metric.compute(predictions=flat_predictions, references=flat_labels, average="weighted")
    recall = recall_metric.compute(predictions=flat_predictions, references=flat_labels, average="weighted")
    f1 = f1_metric.compute(predictions=flat_predictions, references=flat_labels, average="weighted")

    return {
        "accuracy": accuracy["accuracy"],
        "precision": precision["precision"],
        "recall": recall["recall"],
        "f1": f1["f1"],
    }

In [55]:
# Initialize Trainer with BiLSTM model

# Print the size of the training dataset
print(f"Size of training dataset: {len(train_data)}")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

Size of training dataset: 35973


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.996900,1.004649,0.471175,0.432652,0.471175,0.445504
2,0.997800,1.004038,0.471884,0.434962,0.471884,0.444616
3,0.999700,1.001982,0.471695,0.434599,0.471695,0.445352
4,0.994800,1.001244,0.471941,0.434986,0.471941,0.445737


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/m

TrainOutput(global_step=8996, training_loss=0.9985389431724022, metrics={'train_runtime': 185.6489, 'train_samples_per_second': 775.076, 'train_steps_per_second': 48.457, 'total_flos': 0.0, 'train_loss': 0.9985389431724022, 'epoch': 4.0})

In [57]:
from sklearn.metrics import classification_report

# Evaluate the model on the test data using the trainer's evaluate method
eval_results = trainer.evaluate(test_data)

# The eval_results dictionary contains the metrics calculated by compute_metrics, including Q3 accuracy and f1
q3_accuracy = eval_results["eval_accuracy"]
f1_score = eval_results["eval_f1"]

# To get the classification report, we need the flattened, non-padded true and predicted labels from the test set
# We can get the predictions and labels using trainer.predict and then filter
predictions = trainer.predict(test_data)
predicted_labels = np.argmax(predictions.predictions, axis=-1)
true_labels = predictions.label_ids

# Flatten and filter out padding (-100)
flat_predicted_labels = predicted_labels.flatten()
flat_true_labels = true_labels.flatten()

mask = flat_true_labels != -100
filtered_predicted_labels = flat_predicted_labels[mask]
filtered_true_labels = flat_true_labels[mask]


# Print Q3 accuracy and F1 score
print("Q3 Accuracy: {:.4f}".format(q3_accuracy))
print("Weighted F1 Score: {:.4f}".format(f1_score))


# Print classification report
# Define target names based on your label_map
target_names = ["Coil", "Sheet", "Helix"] # Assuming 0: Coil, 1: Sheet, 2: Helix
# Explicitly specify the labels to report on
report_labels = [0, 1, 2] # Corresponding to Coil, Sheet, Helix

print("\nClassification Report:")
print(classification_report(filtered_true_labels, filtered_predicted_labels, labels=report_labels, target_names=target_names))

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Q3 Accuracy: 0.4730
Weighted F1 Score: 0.4467

Classification Report:
              precision    recall  f1-score   support

        Coil       0.56      0.69      0.62    247790
       Sheet       0.40      0.32      0.36    126776
       Helix       0.44      0.37      0.40    168294

    accuracy                           0.50    542860
   macro avg       0.47      0.46      0.46    542860
weighted avg       0.49      0.50      0.49    542860

